In [1]:
import pandas as pd
import numpy as np
import sklearn.metrics as skm
import matplotlib.pyplot as plt
from scipy import stats
import sys
import time
import matplotlib
import xgboost as xgb
import seaborn as s
from lib_submision import *

In [ ]:
t = CEventsTable()
t.appendFromCsv("ivan_ntuples/tH.csv",1,"tH")
t.appendFromCsv("ivan_ntuples/ttb.csv",0,"ttb")
t.appendFromCsv("ivan_ntuples/ttc.csv",0,"ttc")
t.appendFromCsv("ivan_ntuples/ttL.csv",0,"ttL")
t.appendFromCsv("nazim_ntuples/ttH.csv",0,"ttH")
t.appendFromCsv("nazim_ntuples/ttZ.csv",0,"ttZ")
t.appendFromCsv("ivan_ntuples/ttW.csv",0,"ttW")
t.appendFromCsv("ivan_ntuples/tZq.csv",0,"tZq")
t.appendFromCsv("ivan_ntuples/tWZ.csv",0,"tWZ")
t.appendFromCsv("ivan_ntuples/tW.csv",0,"single_tW")
t.appendFromCsv("ivan_ntuples/single_t1.csv",0,"single_tt")
t.appendFromCsv("ivan_ntuples/single_t2.csv",0,"single_tt")
t.appendFromCsv("ivan_ntuples/single_ts.csv",0,"single_ts")
t.appendFromCsv("nazim_ntuples/WZ.csv",0,"WZ")
t.appendFromCsv("nazim_ntuples/VV1.csv",0,"VV")
t.appendFromCsv("nazim_ntuples/VV2.csv",0,"VV")
t.appendFromCsv("ivan_ntuples/data.csv",0,"nonp")

t.subSample('ttH', 0.1)


print('Before preselection:')
t.printTypesNumbers()
t.applyPreselection()
print('After preselection:')
t.printTypesNumbers()
print('With weights:')
t.calculateWeights()
t.printTypesNumbers()


    
mld = CSampledDatasetExpEvents(t.table, (20000,100000),(10000,100000))
#mld = CSampledDatasetAcc(t.table, (20000,100000), (10000,100000))
mld.printInfo()


X_train = mld.X_train
y_train = mld.y_train
types_train = mld.types_train
weights_train = mld.weights_train

X_test = mld.X_test
y_test = mld.y_test
types_test = mld.types_test
weights_test = mld.weights_test


X_val = mld.X_val
y_val = mld.y_val
types_val = mld.types_val
weights_val = mld.weights_val


dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)
dval = xgb.DMatrix(X_val, label=y_val)



AucCalc = CEvaluationAUC()
SigCalc = CEvaluationSignificance()


In [ ]:
# prvni beh - optimalizace gbtree, depth, scale_pos_weight, lambda, alpha, gamma
df1 = pd.read_csv('tpe_sampler/initial_parameters/expected_events/perfs.csv')
df1['df_id'] = 1

# dalsi beh initial parameters - optimalizace lambda, alpha, gamma, min_child_weight, colsample_bytree 
df2 = pd.read_csv('tpe_sampler/perfs.csv')
df2['df_id'] = 2


# nsga algoritmus (optimalizace auc) - optimalizace lambda, alpha, gamma, min_child_weight 
df3 = pd.DataFrame()
for i in range(5):
    _df = pd.read_csv('nsga/df' + str(i) )
    df3 = pd.concat([df3, _df])
df3['df_id'] = 3

    
# nsga algoritmus (optimalizace significance) - optimalizace lambda, alpha, gamma, min_child_weight 
df4 = pd.DataFrame()
for i in range(5):
    _df = pd.read_csv('nsga_sig/df' + str(i) )
    df4 = pd.concat([df4, _df])
df4['df_id'] = 4
    
# tpe (optimalizace sig) - lambda, alpha, gamma, min_child_weight (vetsi rozsahy) 
df5 = pd.DataFrame()
for i in range(2):
    _df = pd.read_csv('tpe_sampler_sig/df' + str(i) )
    df5 = pd.concat([df5, _df])
df5['df_id'] = 5    
    
# tpe (optimalizace sig) - lambda, alpha, gamma, min_child_weight, colsample_bytree 
df6 = pd.DataFrame()
for i in range(4):
    _df = pd.read_csv('tpe_sampler_sig2/df' + str(i) )
    df6 = pd.concat([df6, _df])
df6['df_id'] = 6
    
nsga = pd.concat([df3,df4])
    
dfa = pd.concat([df2,df3,df4,df5,df6])
c = dfa.pop('df_id')
dfa.insert(0, 'df_id', c)

In [ ]:
# # Getting best models from optuna analysis and saving them

models = []

for i in range(20):
    row = dfa.iloc[i,:]
    colsample_bytree = row['colsample_bytree']

    if row['df_id'] == 5:
        colsample_bytree = 0.6    
        
    best_epoch_t = row['best_epoch']

    while 1:
        (model, sig, auc, best_epoch) = evaluate_model(row['lambda'], row['alpha'], row['gamma'],
                       row['min_child_weight'], colsample_bytree)
        
        print(best_epoch, best_epoch_t)
        if best_epoch_t == best_epoch:
            break
        else:
            print('results dont match, running again\n')
    
    models.append([model,sig,auc])

    
    
    
for i in range(len(models)):
    model = models[i][0]
    
    joblib.dump(model, "nn_models/sig_opt/m" + str(i))
    model = joblib.load("nn_models/sig_opt/m" + str(i))
    
    preds = model.predict(X_test)
    auc = AucCalc.evaluate(preds, y_test)
    significance, thr, sig, bg = SigCalc.evaluate(preds, y_test, weights_test)


In [ ]:
models = []

for i in range(20):
    row = dfa.iloc[i,:]
    colsample_bytree = row['colsample_bytree']

    if row['df_id'] == 5:
        colsample_bytree = 0.6    
        
    best_epoch_t = row['best_epoch']

    while 1:
        (model, sig, auc, best_epoch) = evaluate_model(row['lambda'], row['alpha'], row['gamma'],
                       row['min_child_weight'], colsample_bytree)
        
        print(best_epoch, best_epoch_t)
        if best_epoch_t == best_epoch:
            break
        else:
            print('results dont match, running again\n')
    
    models.append([model,sig,auc])

In [ ]:
models_sigopt = []
models_aucopt = []

for i in range(20):
    model = joblib.load("xgb_models/auc_opt/m" + str(i))
    models_aucopt.append(model)
    
    model = joblib.load("xgb_models/sig_opt2/m" + str(i))
    models_sigopt.append(model)
    
not_needed = CSampledDatasetExpEvents(t.table, (20000,100000))
not_needed.printInfo()
no_train = not_needed.X_test

no_train['y'] = not_needed.y_test
no_train['type'] = not_needed.types_test
no_train['weight'] = not_needed.weights_test

In [ ]:
# getting aucs and significance boxplots

n_tests=10

for i in range(n_tests):
    samples = CSampledDatasetExpEvents(no_train, (20000,100000), rs = i)
    X_samples = samples.X_train
    y_samples = samples.y_train
    weights_samples = samples.weights_train
    
    for m in range(len(models_aucopt)):
        mod_auc = models_aucopt[m]

        preds = mod_auc.predict(X_samples)
        auc = AucCalc.evaluate(preds, y_samples, print_score=False)
        significance, thr, sig, bg = SigCalc.evaluate(preds, y_samples, weights_samples, print_score=False)
        
        aucs.loc[i, "auc_opt" + str(m)] = auc
        sigs.loc[i, "auc_opt" + str(m)] = significance
        n_signals.loc[i, "auc_opt" + str(m)] = sig
        
        mod_sig = models_sigopt[m]
        
        preds = mod_sig.predict(X_samples)
        auc = AucCalc.evaluate(preds, y_samples, print_score=False)
        significance, thr, sig, bg = SigCalc.evaluate(preds, y_samples, weights_samples, print_score=False)
        
        aucs.loc[i, "sig_opt" + str(m)] = auc
        sigs.loc[i, "sig_opt" + str(m)] = significance
        n_signals.loc[i, "sig_opt" + str(m)] = sig
    
    display(aucs.head(10))
    display(sigs.head(10))
    display(n_signals.head(10))

In [ ]:
p = sigs.iloc[:, 0:10]


matplotlib.rcParams.update({'font.size': 12,'font.weight':'normal'})


plt.figure(figsize=(12,8))
plt.grid(True)


boxprops = dict(linestyle='-', linewidth=1.5, color='k')
whiskerprops = dict(linestyle='-', linewidth=1.5, color='k')
capprops = dict(linestyle='-', linewidth=1.5, color='k')

plt.xlabel("Model")
plt.ylabel("AUC")

plt.boxplot(p, labels=range(10), 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops)
plt.savefig('diplomka_obrazky/models_sig_boxplots.png',dpi=300)

plt.show()

In [ ]:
# showing multiple significance plots in on graph

for i in range(0,6):    
    samples = CSampledDatasetExpEvents(no_train, (20000,100000), rs = i)
    
    X_samples = samples.X_train
    y_samples = samples.y_train
    weights_samples = samples.weights_train


    model = models_sigopt[1]

    preds = model.predict(X_samples)
    auc = AucCalc.evaluate(preds, y_samples, print_score=True, show_graph=True)
    significance, thr, sig, bg = SigCalc.evaluate(preds, y_samples, weights_samples, 
                                                  print_score=True, show_graph=False, save=True)

In [ ]:
# getting feature importances of different models

df=pd.DataFrame(columns = ['njets_CBT5', 'leptons_charge', 'nbjets', 'chi2_min_tophad_m_ttAll', 'fwdjets_pt', 
                           'nnonbjets', 'sphericity', 'nonbjets_pt', 'nfwdjets', 'chi2_min_DeltaEta_tH', 'chi2_min_bbnonbjet_m', 
                           'chi2_min_higgs_m', 'chi2_min_Whad_m_ttAll', 'inv3jets', 'nonbjets_eta','rapgap_top_fwdjet', 'chi2_min_toplep_pt', 
                           'aplanarity', 'chi2_min_Imvmass_tH', 
                           'tagnonb_topb_m','tagnonb_eta','rapgap_maxptjet','chi2_min_deltaRq1q2','foxWolfram_2_momentum'])

for i in range(20):
    l = pd.DataFrame(models_sigopt[i].model.get_score( importance_type='gain'), index= [i])
    df = pd.concat([df,l])




matplotlib.rcParams.update({'font.size': 13,'font.weight':'normal'})


plt.figure(figsize=(12,8))
plt.grid(True)


boxprops = dict(linestyle='-', linewidth=1.5, color='k')
whiskerprops = dict(linestyle='-', linewidth=1.5, color='k')
capprops = dict(linestyle='-', linewidth=1.5, color='k')


plt.xlabel("Importance rank")
plt.ylabel("Average gain")

plt.boxplot(df, labels=range(24), 
            boxprops=boxprops, whiskerprops=whiskerprops, capprops=capprops)

plt.show()

In [ ]:
# training of different models on a subset of features

def evaluate_model(lambdaa, alpha, gamma, min_child_weight, colsample_bytree):
    booster = 'gbtree'
    max_depth = 5
    scale_pos_weight = 1.0


    model = XGBWrapper()
    model.train(dtrain, 10000, max_depth, 0.1, dval, 
                scale_pos_weight = scale_pos_weight, lambdaa = lambdaa,
                alpha = alpha, gamma = gamma, booster = booster, 
                min_child_weight = min_child_weight, colsample_bytree = colsample_bytree)

    auc_test = model.best_performance
    auc_train = model.best_performance_train
    best_epoch = model.best_epoch
    n_epochs = model.n_epochs
    el_time =  model.elapsed_time

    print('\ntraining:')
    print('auc train:',auc_train,'auc val:', auc_test)
    print('best epoch:', best_epoch, 'epoch cnt:', n_epochs, 
          'time:',el_time)

    preds = model.predict(X_test)
    auc = AucCalc.evaluate(preds, y_test)
    significance, thr, sig, bg = SigCalc.evaluate(preds, y_test, weights_test)
    
    
    preds = model.predict(X_val)
    significance2, thr2, sig2, bg2 = SigCalc.evaluate(preds, y_val, weights_val)


    print('test auc:', auc)
    print('test sig:', significance)

    return model, significance, auc, significance2, auc_test, sig2



for i in range(20):
    mld = CSampledDatasetExpEvents(t.table, (20000,100000), (10000,100000), rs=i)
    #mld = CSampledDatasetAcc(t.table, (20000,100000), (10000,100000))
    mld.printInfo()


    X_train = mld.X_train
    X_train.drop(columns = ['njets_CBT5','inv3jets'], inplace = True)
    y_train = mld.y_train
    types_train = mld.types_train
    weights_train = mld.weights_train

    X_test = mld.X_test
    X_test.drop(columns = ['njets_CBT5','inv3jets'], inplace = True)
    y_test = mld.y_test
    types_test = mld.types_test
    weights_test = mld.weights_test


    X_val = mld.X_val
    X_val.drop(columns = ['njets_CBT5','inv3jets'], inplace = True)
    y_val = mld.y_val
    types_val = mld.types_val
    weights_val = mld.weights_val


    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test, label=y_test)
    dval = xgb.DMatrix(X_val, label=y_val)
    AucCalc = CEvaluationAUC()
    SigCalc = CEvaluationSignificance()
    
    
    
    if i<10:
        r = 0
    else:
        r = 1
            
    row = dfa.iloc[r,:]
    colsample_bytree = row['colsample_bytree']

    print(i,row['colsample_bytree'])

    #if row['df_id'] == 5:
    #    colsample_bytree = 0.6    
        
    best_epoch_t = row['best_epoch']

    (model, significance, auc, sig_val, auc_val, nsig) = evaluate_model(row['lambda'], row['alpha'], row['gamma'],
                   row['min_child_weight'], colsample_bytree)
        
    
    results.append([model, significance, auc, sig_val, auc_val, nsig])
    